In [1]:
import os, numpy, PIL
from PIL import Image
import datetime
import time
import cv2
import os
import traceback
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Run this

In [2]:
def get_date_taken(path):

    try:
        datetime_str = Image.open(path)._getexif()[36867]
        datetime_obj = datetime.datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
    except:
        return datetime.datetime.now()

    return datetime_obj

In [3]:
def compress_images(ip_dir, op_dir, res=None, image_quality=100, ext='.jpg'):
    
    tic = datetime.datetime.now()

    splt = ip_dir.split('/')
    sub_dir = splt[len(splt)-1] + '_compressed'
    op_sub_dir = os.path.join(op_dir, sub_dir)
    
    images = [os.path.join(ip_dir,img) for img in os.listdir(ip_dir) if not img.startswith('.') and img.lower().endswith(ext)]
    
    if (len(images) == 0):
        print('No images found with extension ' + ext)
        return
    
    if res is None:
        frame = cv2.imread(os.path.join(ip_dir, images[0]))
        height, width, layers = frame.shape
        res = (width, height)
    else:
        width, height = res
        
    if not os.path.exists(op_sub_dir):
        os.makedirs(op_sub_dir)
    
    img_counter = 0
    for image in images:
        im = Image.open(image)
        im_resized = im.resize(res, Image.ANTIALIAS)
        
        compressed_file = os.path.join(op_sub_dir, os.path.basename(image))
        im_resized.save(compressed_file, optimize=True, quality=image_quality)
        img_counter += 1
        print('Compressing ('+str(img_counter)+'/'+str(len(images))+') images', end='\r')
        
    print('Compression Completed in ', (datetime.datetime.now()-tic), 'sec(s)')
        
    return op_sub_dir


In [4]:
def create_video (ip_dir, op_dir, res, frame_rate=24, every_nth_image=1, 
                  image_quality=100, frames_per_video=-1, 
                  ext='.jpg', sort_date=False):
    
    tic = datetime.datetime.now()

    splt = ip_dir.split('/')
    sub_dir = splt[len(splt)-1]
    
    images = [img for img in os.listdir(ip_dir) if not img.startswith('.') and img.lower().endswith(ext)]
    modified_dates = []
    date_available_flag = True
    
    for image in images:
        # image = os.path.join(ip_dir, image)
        mod_date = get_date_taken(image)
        if (mod_date == None):
            date_available_flag = False
            break
        modified_dates.append(mod_date)

    print('Image count =', len(images), end=' : ')
    if (len(images) == 0):
        print('No images found with extension ' + ext)
        return
    
    df_data = pd.DataFrame(list(zip(images, modified_dates)), columns=['filename', 'image_date'])
       
    if (sort_date):
        df_data = df_data.sort_values(by=['image_date'], ascending=True)
    else:
        df_data = df_data.sort_values(by=['filename'], ascending=True)
        
    df_data['filename'] = df_data['filename'].apply(lambda x: os.path.join(ip_dir, x))

    images = df_data['filename']
    
    if res is None:
        frame = cv2.imread(os.path.join(ip_dir, images[0]))
        height, width, layers = frame.shape
        res = (width, height)
    else:
        width, height = res
        
    print('Height:',height,'Width:', width)

    temp_file = 'temp.jpg'
    count = 0
    video_part_count = 0
    video = None
    
    if (frames_per_video == -1):
        frames_per_video = len(images)
        
    for image in images:

        try:
            
            if (count%frames_per_video == 0):
                video_part_count += 1
                op_file = os.path.join(op_dir, sub_dir+'_'+str(frame_rate)+'fps_'+str(width)+'x'+str(height)\
                                       +'_part'+str(video_part_count)+'.avi')
                print('Output Video: ', op_file)
                video = cv2.VideoWriter(op_file, 0, frame_rate, res)
                
            if (count%every_nth_image == 0):
                image = os.path.join(ip_dir, image)

                im = Image.open(image)
                im_resized = im.resize(res, Image.ANTIALIAS)
                im_resized.save(temp_file, optimize=True, quality=image_quality)

                video.write(cv2.imread(temp_file))
                # print(image)

            count += 1
            print('Frames ('+str(count)+'/'+str(len(images))+') Elapsed Time:',str(datetime.datetime.now()-tic), end='\r')
        except:
            traceback.print_exc()
            continue

    if os.path.exists(temp_file):
        os.remove(temp_file)

    cv2.destroyAllWindows()
    video.release()

In [5]:
INPUT_DIR = '/Volumes/Mac Hawaii/Photos/Timelapse_'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

INPUT_DIR = '/Users/talat/Pictures/_Photos/Edited/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

INPUT_DIR = '/Volumes/PhotosAug20/Photos/Edited/'
OUTPUT_DIR = '/Volumes/PhotosAug20/Photos/Video/'

INPUT_DIR = '/Users/talat/Pictures/_Photos/Edited/'
INTERMEDIATE_DIR = '/Users/talat/Pictures/_Photos/Timelapse_compressed/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

# INPUT_DIR = '/Volumes/TOSHIBA/Photos/Timelapse Edited 1MB/'
# OUTPUT_DIR = '/Volumes/TOSHIBA/Photos/Videos OG/'

# INPUT_DIR = '/Users/talat/Pictures/_Photos/T1/'
# OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'
# OUTPUT_DIR = '/Volumes/OG 2/TL Videos Uncompressed/'

INPUT_DIR = '/Users/talat/Pictures/_Photos/Timelapse Edited/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/'

# INPUT_DIR = '/Volumes/5TB/Photos/Timelapse Edited/'
# OUTPUT_DIR = '/Volumes/5TB/Photos/Timelapse Videos Uncompressed/'
# OUTPUT_DIR = '/Volumes/New Volume/TL Videos Uncompressed 1/'
# OUTPUT_DIR = '/Volumes/Empty/Timelapse Videos Uncompressed - 2/' 
# OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/'

res_hd = (1920, 1080)
res_4k = (3840, 2160)
res = res_hd
res = None

every_nth_image = 1
frame_rate = 24
ext = '.jpg'
image_quality = 25
compress = False
number_of_seconds = 20
frames_per_video = frame_rate * number_of_seconds

# Create output & intermediate directories
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
if not os.path.exists(INTERMEDIATE_DIR):
    os.makedirs(INTERMEDIATE_DIR)

sub_dir_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR,f))]
sub_dir_list.sort()

print('Number of sub-directories found: '+str(len(sub_dir_list)))
count = 1

for sub_dir in sub_dir_list:
    
    try:
        tic = datetime.datetime.now()
        ip_dir = os.path.join(INPUT_DIR, sub_dir)
        op_dir = OUTPUT_DIR
        int_dir = INTERMEDIATE_DIR
        
        # print(ip_dir, '\n', op_dir, '\n', int_dir)

        print('['+str(count)+'/'+str(len(sub_dir_list))+'] '+sub_dir, end=' : ')
        
        if compress:
            compressed_dir = compress_images(ip_dir=ip_dir, op_dir=int_dir, image_quality=image_quality, ext=ext)
            ip_dir = compressed_dir

        create_video(ip_dir=ip_dir, op_dir=op_dir, res=res, frame_rate=frame_rate, image_quality=image_quality,
                     every_nth_image=every_nth_image, frames_per_video=frames_per_video, ext=ext)
        
        # create_video(ip_dir=ip_dir, op_dir=op_dir, res=res, frame_rate=frame_rate, every_nth_image=every_nth_image, ext=ext)
        
        print('Video created in ', str(datetime.datetime.now()-tic), 'seconds at ', datetime.datetime.now())
    except:
        print('EXCEPTION: having issues with the directory: '+sub_dir)
        traceback.print_exc()
        continue
    
    count += 1

Number of sub-directories found: 6
[1/6] 2022-04-04 - Sunset (TL) E : Image count = 273 : Height: 4000 Width: 6000
Output Video:  /Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/2022-04-04 - Sunset (TL) E_24fps_6000x4000_part1.avi
Video created in  0:03:04.167879 seconds at  2022-04-21 21:36:14.656795
[2/6] 2022-04-07 - Sunset (TL) E : Image count = 181 : Height: 4000 Width: 6000
Output Video:  /Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/2022-04-07 - Sunset (TL) E_24fps_6000x4000_part1.avi
Video created in  0:03:06.228833 seconds at  2022-04-21 21:39:20.885761
[3/6] 2022-04-08 - Sunset (TL) E : Image count = 1198 : Height: 4000 Width: 6000
Output Video:  /Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/2022-04-08 - Sunset (TL) E_24fps_6000x4000_part1.avi
Output Video:  /Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/2022-04-08 - Sunset (TL) E_24fps_6000x4000_part2.avi
Output Video:  /Users/talat/Pictures/_Photos/Timelapse Videos Uncomp

# Compress Video

In [ ]:
!pip install ffmpeg-python

In [ ]:
## Something is not working in compression

In [ ]:
import os, ffmpeg

# Code from: https://stackoverflow.com/questions/10607468/how-to-reduce-the-image-file-size-using-pil
def compress_video(video_full_path, output_file_name, target_size):
    # Reference: https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
    min_audio_bitrate = 32000
    max_audio_bitrate = 256000

    probe = ffmpeg.probe(video_full_path)
    # Video duration, in s.
    duration = float(probe['format']['duration'])
    # Audio bitrate, in bps.
    audio_bitrate = float(next((s for s in probe['streams'] if s['codec_type'] == 'audio'), None)['bit_rate'])
    # Target total bitrate, in bps.
    target_total_bitrate = (target_size * 1024 * 8) / (1.073741824 * duration)

    # Target audio bitrate, in bps
    if 10 * audio_bitrate > target_total_bitrate:
        audio_bitrate = target_total_bitrate / 10
        if audio_bitrate < min_audio_bitrate < target_total_bitrate:
            audio_bitrate = min_audio_bitrate
        elif audio_bitrate > max_audio_bitrate:
            audio_bitrate = max_audio_bitrate
    # Target video bitrate, in bps.
    video_bitrate = target_total_bitrate - audio_bitrate

    i = ffmpeg.input(video_full_path)
    ffmpeg.output(i, os.devnull,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 1, 'f': 'mp4'}
                  ).overwrite_output().run()
    ffmpeg.output(i, output_file_name,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 2, 'c:a': 'aac', 'b:a': audio_bitrate}
                  ).overwrite_output().run()

In [ ]:
import os, ffmpeg

# Code from: https://stackoverflow.com/questions/10607468/how-to-reduce-the-image-file-size-using-pil
def compress_video(video_full_path, output_file_name, target_size):
    # Reference: https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
    
    # min_audio_bitrate = 32000
    # max_audio_bitrate = 256000

    probe = ffmpeg.probe(video_full_path)
    # Video duration, in s.
    duration = float(probe['format']['duration'])
    
    print(probe['streams'])
    
    # Audio bitrate, in bps.
    # audio_bitrate = float(next((s for s in probe['streams'] if s['codec_type'] == 'audio'), None)['bit_rate'])
    
    # Target total bitrate, in bps.
    target_total_bitrate = (target_size * 1024 * 8) / (1.073741824 * duration)

    audio_bitrate = 0
    # Target audio bitrate, in bps
    if 10 * audio_bitrate > target_total_bitrate:
        audio_bitrate = target_total_bitrate / 10
        if audio_bitrate < min_audio_bitrate < target_total_bitrate:
            audio_bitrate = min_audio_bitrate
        elif audio_bitrate > max_audio_bitrate:
            audio_bitrate = max_audio_bitrate
    # Target video bitrate, in bps.
    video_bitrate = target_total_bitrate - audio_bitrate

    i = ffmpeg.input(video_full_path)
    ffmpeg.output(i, os.devnull,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 1, 'f': 'mp4'}
                  ).overwrite_output().run()
    ffmpeg.output(i, output_file_name,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 2, 'c:a': 'aac', 'b:a': audio_bitrate}
                  ).overwrite_output().run()

In [ ]:
ip_vid = '/Users/talat/Pictures/_Photos/Video/2021-11-09 - Sunset 2 T6i DD (TL) E_24fps_6000x4000.avi'
op_vid = '/Users/talat/Pictures/_Photos/Video/2021-11-09 - Sunset 2 T6i DD (TL) E_24fps_6000x4000_comp.avi'
compress_video(ip_vid, op_vid, 50 * 1000)

# The following cell is only for testing compression

In [ ]:
from PIL import Image
 # My image is a 200x374 jpeg that is 102kb large
foo = Image.open("../data/compression/sample.jpg")
width, height = foo.size
quality = 25

print(width, height)

 # I downsize the image with an ANTIALIAS filter (gives the highest quality)
foo = foo.resize((width,height),Image.ANTIALIAS)

foo.save("../data/compression/sample_compressed.jpg",quality=quality)
# The saved downsized image size is 24.8kb

foo.save("../data/compression/sample_optimized.jpg",optimize=True,quality=quality)
# The saved downsized image size is 22.9kb

In [ ]:
ip_dir = "../data/compression/"
splt = ip_dir.split('/')
sub_dir = splt[len(splt)-2] + '_compressed'

print(sub_dir)